In [420]:
import yfinance as yf
import numpy as np
import pandas as pd
import talib
import pandas_datareader.data as pdr
import statsmodels.api as sm
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
#plt.style.use('seaborn-white')
plt.rcParams["figure.figsize"] = (20,10)
#plt.rcParams['axes.facecolor'] = 'grey'
#myplot + theme(panel.background = element_rect(fill='green', colour='red'))

## Choosing start and end dates, and Portfolio whose performance is to be measured

In [421]:
end_date = dt.date(2020, 1, 1)
start_date = dt.date(end_date.year - 5, end_date.month, end_date.day)

fund1 = yf.Ticker("JUNIORBEES.NS")
print(start_date)
print(end_date)

2015-01-01
2020-01-01


Getting Portfolio data using yf and changing the dataframe to get monthly returns

In [422]:
dft1 = fund1.history(start = start_date, end = end_date, interval = "1mo")["Close"]
dft1 = pd.DataFrame(dft1)
#dft1 = dft1[1:]
dft1
#dft1 = dft1[1:]


,Close
Date,
2015-01-01,196.679993
2015-02-01,197.773605
2015-03-01,194.009995
2015-04-01,192.800003
2015-05-01,198.300003
...,...
2019-09-01,285.540009
2019-10-01,296.700012
2019-11-01,295.750000


In [423]:
dft1 = pd.DataFrame(dft1["Close"].pct_change()*100)
dft1 = dft1[1:]
print(dft1.shape)
dft1 = dft1.rename(columns = {"Close" : "Monthly Returns in %"})
dft1

(60, 1)


,Monthly Returns in %
Date,
2015-02-01,0.556037
2015-03-01,-1.902989
2015-04-01,-0.623675
2015-05-01,2.852697
2015-06-01,-0.690878
2015-07-01,6.418532
2015-08-01,-3.411752
2015-09-01,-2.376246
2015-10-01,0.703405


## Using IIM-A resource to get 6 value weight portfolios and resampling it to a dataframe

In [424]:
dfp = pd.read_csv("https://faculty.iima.ac.in/~iffm/Indian-Fama-French-Momentum/DATA/20191231_Size_Value_Portfolio_Returns_Daily.csv")
dfp["Date"] = pd.to_datetime(dfp["Date"], format = "%Y%m%d")
dfp.set_index("Date", inplace = True)
dfp = dfp/100
dfpm = dfp.resample('M').agg(lambda x: (x + 1).prod() - 1)
#mean()
#agg({"Portfolio SN returns %": lambda x: ((x + 1).prod() - 1)})
#dfp = dfp.iloc[195:]
dfpm = dfpm*100
dfpm = dfpm.iloc[-60:]
dfpm

,Portfolio BV returns %,Portfolio BN returns %,Portfolio BG returns %,Portfolio SV returns %,Portfolio SN returns %,Portfolio SG returns %
Date,,,,,,
2015-01-31,5.159860,0.267589,6.317358,2.939539,3.829209,2.092794
2015-02-28,-3.598628,-1.620544,1.725937,-0.561409,-0.036486,-0.025230
2015-03-31,-12.067794,-6.001506,-2.216403,-7.390002,-4.558309,-0.939429
2015-04-30,1.987220,0.404376,-3.462755,1.496758,1.920260,-0.672204
2015-05-31,-2.266211,0.144783,4.182844,-1.318443,2.446716,2.702724
2015-06-30,-15.038829,-3.504739,-0.581203,-3.454943,-0.149459,-2.086199
2015-07-31,1.611151,3.681677,2.385610,15.968174,12.552245,9.268494
2015-08-31,-21.041072,-9.126825,-5.587638,-10.542894,-8.371771,-7.295656
2015-09-30,1.315859,0.761272,-0.812666,2.277109,1.179906,0.602886


## Calculating SMB and HML

In [425]:
dfpm["SMB in %"] = (dfpm["Portfolio SV returns %"] + dfpm["Portfolio SN returns %"] + dfpm["Portfolio SG returns %"] - dfpm["Portfolio BV returns %"] - dfpm["Portfolio BN returns %"] - dfpm["Portfolio BG returns %"])/3
dfpm["HFL in %"] = (dfpm["Portfolio SV returns %"] + dfpm["Portfolio BV returns %"] - dfpm["Portfolio SG returns %"] - dfpm["Portfolio BG returns %"])/2
dfpm

,Portfolio BV returns %,Portfolio BN returns %,Portfolio BG returns %,Portfolio SV returns %,Portfolio SN returns %,Portfolio SG returns %,SMB in %,HFL in %
Date,,,,,,,,
2015-01-31,5.159860,0.267589,6.317358,2.939539,3.829209,2.092794,-0.961088,-0.155377
2015-02-28,-3.598628,-1.620544,1.725937,-0.561409,-0.036486,-0.025230,0.956703,-2.930372
2015-03-31,-12.067794,-6.001506,-2.216403,-7.390002,-4.558309,-0.939429,2.465988,-8.150982
2015-04-30,1.987220,0.404376,-3.462755,1.496758,1.920260,-0.672204,1.271991,3.809469
2015-05-31,-2.266211,0.144783,4.182844,-1.318443,2.446716,2.702724,0.589860,-5.235111
2015-06-30,-15.038829,-3.504739,-0.581203,-3.454943,-0.149459,-2.086199,4.478057,-7.913185
2015-07-31,1.611151,3.681677,2.385610,15.968174,12.552245,9.268494,10.036825,2.962611
2015-08-31,-21.041072,-9.126825,-5.587638,-10.542894,-8.371771,-7.295656,3.181738,-9.350336
2015-09-30,1.315859,0.761272,-0.812666,2.277109,1.179906,0.602886,0.931812,1.901374


### dff is the final dataframe to be analyzed

In [426]:
dft1.index = dfpm.index
dff = pd.merge(dft1, dfpm, on = "Date")
dff

,Monthly Returns in %,Portfolio BV returns %,Portfolio BN returns %,Portfolio BG returns %,Portfolio SV returns %,Portfolio SN returns %,Portfolio SG returns %,SMB in %,HFL in %
Date,,,,,,,,,
2015-01-31,0.556037,5.159860,0.267589,6.317358,2.939539,3.829209,2.092794,-0.961088,-0.155377
2015-02-28,-1.902989,-3.598628,-1.620544,1.725937,-0.561409,-0.036486,-0.025230,0.956703,-2.930372
2015-03-31,-0.623675,-12.067794,-6.001506,-2.216403,-7.390002,-4.558309,-0.939429,2.465988,-8.150982
2015-04-30,2.852697,1.987220,0.404376,-3.462755,1.496758,1.920260,-0.672204,1.271991,3.809469
2015-05-31,-0.690878,-2.266211,0.144783,4.182844,-1.318443,2.446716,2.702724,0.589860,-5.235111
2015-06-30,6.418532,-15.038829,-3.504739,-0.581203,-3.454943,-0.149459,-2.086199,4.478057,-7.913185
2015-07-31,-3.411752,1.611151,3.681677,2.385610,15.968174,12.552245,9.268494,10.036825,2.962611
2015-08-31,-2.376246,-21.041072,-9.126825,-5.587638,-10.542894,-8.371771,-7.295656,3.181738,-9.350336
2015-09-30,0.703405,1.315859,0.761272,-0.812666,2.277109,1.179906,0.602886,0.931812,1.901374


## Getting excess index returns and risk free rate data 

In [427]:
df_rfrm = pd.read_csv("https://faculty.iima.ac.in/~iffm/Indian-Fama-French-Momentum/DATA/20191231_FourFactors_and_Market_Returns_Monthly.csv")
df_rfrm = pd.DataFrame(df_rfrm[["Rm-Rf %", "Rf %"]])
df_rfrm = df_rfrm.iloc[-60:]
df_rfrm.index = dff.index
dff = pd.merge(dff, df_rfrm, on = "Date")

Preparing the final dataframe to analyze

In [428]:
dff["Returns - Rf"] = dff["Monthly Returns in %"] - dff["Rf %"]
dff

,Monthly Returns in %,Portfolio BV returns %,Portfolio BN returns %,Portfolio BG returns %,Portfolio SV returns %,Portfolio SN returns %,Portfolio SG returns %,SMB in %,HFL in %,Rm-Rf %,Rf %,Returns - Rf
Date,,,,,,,,,,,,
2015-01-31,0.556037,5.159860,0.267589,6.317358,2.939539,3.829209,2.092794,-0.961088,-0.155377,4.376395,0.673395,-0.117359
2015-02-28,-1.902989,-3.598628,-1.620544,1.725937,-0.561409,-0.036486,-0.025230,0.956703,-2.930372,0.402868,0.639769,-2.542758
2015-03-31,-0.623675,-12.067794,-6.001506,-2.216403,-7.390002,-4.558309,-0.939429,2.465988,-8.150982,-3.510824,0.671909,-1.295584
2015-04-30,2.852697,1.987220,0.404376,-3.462755,1.496758,1.920260,-0.672204,1.271991,3.809469,-3.161306,0.580249,2.272448
2015-05-31,-0.690878,-2.266211,0.144783,4.182844,-1.318443,2.446716,2.702724,0.589860,-5.235111,2.780651,0.612949,-1.303827
2015-06-30,6.418532,-15.038829,-3.504739,-0.581203,-3.454943,-0.149459,-2.086199,4.478057,-7.913185,-1.703791,0.655885,5.762647
2015-07-31,-3.411752,1.611151,3.681677,2.385610,15.968174,12.552245,9.268494,10.036825,2.962611,2.728233,0.670114,-4.081866
2015-08-31,-2.376246,-21.041072,-9.126825,-5.587638,-10.542894,-8.371771,-7.295656,3.181738,-9.350336,-6.835698,0.601800,-2.978046
2015-09-30,0.703405,1.315859,0.761272,-0.812666,2.277109,1.179906,0.602886,0.931812,1.901374,-1.027966,0.575600,0.127805


## Running Multiple Regression

In [429]:
y = dff["Returns - Rf"]
X = dff[["Rm-Rf %", "SMB in %", "HFL in %"]]

X_constant = sm.add_constant(X)

C:\Users\amaan\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [430]:
regression_model = sm.OLS(y, X_constant)
results = regression_model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Returns - Rf   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.046
Method:                 Least Squares   F-statistic:                    0.1422
Date:                Mon, 23 Aug 2021   Prob (F-statistic):              0.934
Time:                        01:04:42   Log-Likelihood:                -173.76
No. Observations:                  60   AIC:                             355.5
Df Residuals:                      56   BIC:                             363.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1984      0.598      0.332      0.741      -1.000       1.397
Rm-Rf %        0.0094      0.170      0.056      0.956      -0.331       0.350
SMB in %       0.0336      0.167      0.201      0.842      -0.301       0.368
HFL in %      -0.0639      0.121     -0.528      0.600      -0.306       0.178
==============================================================================
Omnibus:                        1.616   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.446   Jarque-Bera (JB):                1.017
Skew:                          -0.300   Prob(JB):                        0.601
Kurtosis:                       3.217   Cond. No.                         6.25
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [438]:
dfpm1 = pd.read_csv("https://faculty.iima.ac.in/~iffm/Indian-Fama-French-Momentum/DATA/20191231_FourFactors_and_Market_Returns_Monthly.csv")
dfpm1 = dfpm1.iloc[-60:]
dfpm1.index = dfpm.index
dfpm1 = dfpm1.drop(columns = ["Month", "WML %"])
dfpm1
dff1 = pd.merge(dfpm1, dft1, on = "Date")
dff1["Returns - Rf %"] = dff1["Monthly Returns in %"] - dff1["Rf %"]
dff1

,SMB %,HML %,Rm %,Rf %,Rm-Rf %,Monthly Returns in %,Returns - Rf %
Date,,,,,,,
2015-01-31,-1.015842,-0.033052,5.077839,0.673395,4.376395,0.556037,-0.117359
2015-02-28,0.802264,-2.760060,1.045101,0.639769,0.402868,-1.902989,-2.542758
2015-03-31,2.675235,-8.198993,-2.861378,0.671909,-3.510824,-0.623675,-1.295584
2015-04-30,1.323355,4.044645,-2.598459,0.580249,-3.161306,2.852697,2.272448
2015-05-31,0.480136,-4.937247,3.409817,0.612949,2.780651,-0.690878,-1.303827
2015-06-30,4.853406,-8.006861,-1.058528,0.655885,-1.703791,6.418532,5.762647
2015-07-31,9.665542,2.731236,3.415833,0.670114,2.728233,-3.411752,-4.081866
2015-08-31,3.783760,-9.540706,-6.273030,0.601800,-6.835698,-2.376246,-2.978046
2015-09-30,0.870640,2.057171,-0.457950,0.575600,-1.027966,0.703405,0.127805


In [441]:
y1 = dff1["Returns - Rf %"]
X1 = dff1[["Rm-Rf %", "SMB %", "HML %"]]

X_constant1 = sm.add_constant(X)
regression_model = sm.OLS(y1, X_constant1)
results = regression_model.fit()
results.summary()

C:\Users\amaan\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Returns - Rf %   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                 -0.046
Method:                 Least Squares   F-statistic:                    0.1422
Date:                Mon, 23 Aug 2021   Prob (F-statistic):              0.934
Time:                        01:19:21   Log-Likelihood:                -173.76
No. Observations:                  60   AIC:                             355.5
Df Residuals:                      56   BIC:                             363.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1984      0.598      0.332      0.741      -1.000       1.397
Rm-Rf %        0.0094      0.170      0.056      0.956      -0.331       0.350
SMB in %       0.0336      0.167      0.201      0.842      -0.301       0.368
HFL in %      -0.0639      0.121     -0.528      0.600      -0.306       0.178
==============================================================================
Omnibus:                        1.616   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.446   Jarque-Bera (JB):                1.017
Skew:                          -0.300   Prob(JB):                        0.601
Kurtosis:                       3.217   Cond. No.                         6.25
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""